In [1]:
from tools import *
from data import *
from model_nn import *

c:\SUTD\KTH\DD2421 Machine Learning\DD2421-programming-challenge\data.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[bool_cols] = X_train[bool_cols].astype(int)
c:\SUTD\KTH\DD2421 Machine Learning\DD2421-programming-challenge\data.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = le.fit_transform(X_train[col])


In [2]:
train_kcv(model_type='nn', X_train=X_train, y_train=y_train)

Epoch 1/20


c:\SUTD\KTH\DD2421 Machine Learning\DD2421-programming-challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4532 - loss: 1.0456 - val_accuracy: 0.6050 - val_loss: 0.8481
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5667 - loss: 0.8840 - val_accuracy: 0.6390 - val_loss: 0.7993
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6251 - loss: 0.8323 - val_accuracy: 0.6620 - val_loss: 0.7666
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6274 - loss: 0.8066 - val_accuracy: 0.6660 - val_loss: 0.7478
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6375 - loss: 0.7878 - val_accuracy: 0.6860 - val_loss: 0.7279
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6540 - loss: 0.7670 - val_accuracy: 0.6740 - val_loss: 0.7271
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6481 - loss: 0.7735 - val_accuracy: 0.6840 - val_loss: 0.7203
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6628 - loss: 0.7501 - val_accuracy: 0.6860 - val_

c:\SUTD\KTH\DD2421 Machine Learning\DD2421-programming-challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4733 - loss: 1.0465 - val_accuracy: 0.6070 - val_loss: 0.8670
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5496 - loss: 0.9040 - val_accuracy: 0.6130 - val_loss: 0.8219
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6100 - loss: 0.8410 - val_accuracy: 0.6530 - val_loss: 0.7718
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6376 - loss: 0.7987 - val_accuracy: 0.6690 - val_loss: 0.7486
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6469 - loss: 0.7784 - val_accuracy: 0.6850 - val_loss: 0.7400
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6626 - loss: 0.7674 - val_accuracy: 0.6850 - val_loss: 0.7316
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6555 - loss: 0.7634 - val_accuracy: 0.6900 - val_loss: 0.7245
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6781 - loss: 0.7490 - val_accuracy: 0.6930 - val_

...

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4329 - loss: 1.0722 - val_accuracy: 0.6170 - val_loss: 0.8507
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5852 - loss: 0.8835 - val_accuracy: 0.6350 - val_loss: 0.7988
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6202 - loss: 0.8368 - val_accuracy: 0.6590 - val_loss: 0.7703
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6337 - loss: 0.8095 - val_accuracy: 0.6480 - val_loss: 0.7562
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6515 - loss: 0.7762 - val_accuracy: 0.6560 - val_loss: 0.7472
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6608 - loss: 0.7615 - val_accuracy: 0.6570 - val_loss: 0.7419
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6633 - loss: 0.7641 - val_accuracy: 0.6590 - val_loss: 0.7378
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6710 - loss: 0.7377 - val_accuracy: 0.6540 - val_

c:\SUTD\KTH\DD2421 Machine Learning\DD2421-programming-challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4538 - loss: 1.0482 - val_accuracy: 0.6050 - val_loss: 0.8637
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5924 - loss: 0.8630 - val_accuracy: 0.6250 - val_loss: 0.8090
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6231 - loss: 0.8183 - val_accuracy: 0.6340 - val_loss: 0.7892
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6273 - loss: 0.8004 - val_accuracy: 0.6420 - val_loss: 0.7772
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6713 - loss: 0.7605 - val_accuracy: 0.6520 - val_loss: 0.7632
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6519 - loss: 0.7711 - val_accuracy: 0.6540 - val_loss: 0.7576
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6869 - loss: 0.7276 - val_accuracy: 0.6530 - val_loss: 0.7525
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6750 - loss: 0.7401 - val_accuracy: 0.6590 - val_

c:\SUTD\KTH\DD2421 Machine Learning\DD2421-programming-challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4312 - loss: 1.0948 - val_accuracy: 0.6310 - val_loss: 0.8602
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5832 - loss: 0.8817 - val_accuracy: 0.6590 - val_loss: 0.7884
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6298 - loss: 0.8210 - val_accuracy: 0.6670 - val_loss: 0.7578
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6309 - loss: 0.8147 - val_accuracy: 0.6740 - val_loss: 0.7395
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6459 - loss: 0.8052 - val_accuracy: 0.6750 - val_loss: 0.7270
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6597 - loss: 0.7828 - val_accuracy: 0.6880 - val_loss: 0.7169
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6506 - loss: 0.7810 - val_accuracy: 0.6890 - val_loss: 0.7111
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6860 - loss: 0.7419 - val_accuracy: 0.6880 - val_

In [3]:
train_kcv(model_type='xgb', X_train=X_train, y_train=y_train)

Cross-validation accuracy: 0.6752 ± 0.0157
Cross-validation log loss: 0.7707 ± 0.0195


In [4]:
train_kcv(model_type='gnb', X_train=X_train, y_train=y_train)

Cross-validation accuracy: 0.6500 ± 0.0087
Cross-validation log loss: 0.7565 ± 0.0128
